# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 1 2022 12:16PM,238538,16,SSF-8091747,Sartorius Stedim Filters Inc.,Released
1,Jun 1 2022 12:02PM,238537,19,DEX0006888,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
2,Jun 1 2022 12:02PM,238537,19,DEX0006889,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
3,Jun 1 2022 12:02PM,238537,19,DEX0006890,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
4,Jun 1 2022 12:02PM,238537,19,DEX0006891,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
5,Jun 1 2022 12:02PM,238537,19,DEX0006892,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
6,Jun 1 2022 12:02PM,238537,19,DEX0006893,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
7,Jun 1 2022 12:02PM,238537,19,DEX0006894,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
8,Jun 1 2022 12:02PM,238537,19,DEX0006895,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
9,Jun 1 2022 12:02PM,238537,19,DEX0006896,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
49,238533,Released,1
50,238535,Released,1
51,238536,Executing,1
52,238537,Released,12
53,238538,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238533,NaN,NaN,1.0
238535,NaN,NaN,1.0
238536,NaN,1.0,NaN
238537,NaN,NaN,12.0
238538,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238318,0.0,10.0,10.0
238342,0.0,1.0,0.0
238343,0.0,0.0,1.0
238433,0.0,0.0,23.0
238436,0.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238318,0,10,10
238342,0,1,0
238343,0,0,1
238433,0,0,23
238436,0,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238318,0,10,10
1,238342,0,1,0
2,238343,0,0,1
3,238433,0,0,23
4,238436,0,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238318,,10,10
1,238342,,1,
2,238343,,,1
3,238433,,,23
4,238436,,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 1 2022 12:16PM,238538,16,Sartorius Stedim Filters Inc.
1,Jun 1 2022 12:02PM,238537,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
13,Jun 1 2022 12:00PM,238536,20,Else Nutrition
14,Jun 1 2022 11:57AM,238535,16,Sartorius Stedim Filters Inc.
15,Jun 1 2022 11:41AM,238533,102,"Senseonics, Incorporated"
16,Jun 1 2022 11:09AM,238531,10,Bio-PRF
17,Jun 1 2022 11:08AM,238529,12,Hush Hush
77,Jun 1 2022 11:06AM,238528,10,Bio-PRF
93,Jun 1 2022 10:54AM,238530,10,ISDIN Corporation
94,Jun 1 2022 10:46AM,238526,16,"SD Head USA, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 1 2022 12:16PM,238538,16,Sartorius Stedim Filters Inc.,,,1
1,Jun 1 2022 12:02PM,238537,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,,,12
2,Jun 1 2022 12:00PM,238536,20,Else Nutrition,,1,
3,Jun 1 2022 11:57AM,238535,16,Sartorius Stedim Filters Inc.,,,1
4,Jun 1 2022 11:41AM,238533,102,"Senseonics, Incorporated",,,1
5,Jun 1 2022 11:09AM,238531,10,Bio-PRF,,,1
6,Jun 1 2022 11:08AM,238529,12,Hush Hush,,,60
7,Jun 1 2022 11:06AM,238528,10,Bio-PRF,,,16
8,Jun 1 2022 10:54AM,238530,10,ISDIN Corporation,,,1
9,Jun 1 2022 10:46AM,238526,16,"SD Head USA, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 1 2022 12:16PM,238538,16,Sartorius Stedim Filters Inc.,1,,
1,Jun 1 2022 12:02PM,238537,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,12,,
2,Jun 1 2022 12:00PM,238536,20,Else Nutrition,,1,
3,Jun 1 2022 11:57AM,238535,16,Sartorius Stedim Filters Inc.,1,,
4,Jun 1 2022 11:41AM,238533,102,"Senseonics, Incorporated",1,,
5,Jun 1 2022 11:09AM,238531,10,Bio-PRF,1,,
6,Jun 1 2022 11:08AM,238529,12,Hush Hush,60,,
7,Jun 1 2022 11:06AM,238528,10,Bio-PRF,16,,
8,Jun 1 2022 10:54AM,238530,10,ISDIN Corporation,1,,
9,Jun 1 2022 10:46AM,238526,16,"SD Head USA, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 1 2022 12:16PM,238538,16,Sartorius Stedim Filters Inc.,1,,
1,Jun 1 2022 12:02PM,238537,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,12,,
2,Jun 1 2022 12:00PM,238536,20,Else Nutrition,,1,
3,Jun 1 2022 11:57AM,238535,16,Sartorius Stedim Filters Inc.,1,,
4,Jun 1 2022 11:41AM,238533,102,"Senseonics, Incorporated",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 1 2022 12:16PM,238538,16,Sartorius Stedim Filters Inc.,1.0,NaN,NaN
1,Jun 1 2022 12:02PM,238537,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,12.0,NaN,NaN
2,Jun 1 2022 12:00PM,238536,20,Else Nutrition,NaN,1.0,NaN
3,Jun 1 2022 11:57AM,238535,16,Sartorius Stedim Filters Inc.,1.0,NaN,NaN
4,Jun 1 2022 11:41AM,238533,102,"Senseonics, Incorporated",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 1 2022 12:16PM,238538,16,Sartorius Stedim Filters Inc.,1.0,0.0,0.0
1,Jun 1 2022 12:02PM,238537,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,12.0,0.0,0.0
2,Jun 1 2022 12:00PM,238536,20,Else Nutrition,0.0,1.0,0.0
3,Jun 1 2022 11:57AM,238535,16,Sartorius Stedim Filters Inc.,1.0,0.0,0.0
4,Jun 1 2022 11:41AM,238533,102,"Senseonics, Incorporated",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2862065,121.0,9.0,11.0
102,953920,3.0,1.0,0.0
12,477037,60.0,2.0,0.0
15,1669050,61.0,43.0,0.0
16,954123,4.0,0.0,0.0
18,715207,2.0,1.0,0.0
19,715557,24.0,11.0,6.0
20,1430966,26.0,47.0,16.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2862065,121.0,9.0,11.0
1,102,953920,3.0,1.0,0.0
2,12,477037,60.0,2.0,0.0
3,15,1669050,61.0,43.0,0.0
4,16,954123,4.0,0.0,0.0
5,18,715207,2.0,1.0,0.0
6,19,715557,24.0,11.0,6.0
7,20,1430966,26.0,47.0,16.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,121.0,9.0,11.0
1,102,3.0,1.0,0.0
2,12,60.0,2.0,0.0
3,15,61.0,43.0,0.0
4,16,4.0,0.0,0.0
5,18,2.0,1.0,0.0
6,19,24.0,11.0,6.0
7,20,26.0,47.0,16.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,121.0
1,102,Released,3.0
2,12,Released,60.0
3,15,Released,61.0
4,16,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,18,19,20
Status,,,,,,,,
Completed,11.0,0.0,0.0,0.0,0.0,0.0,6.0,16.0
Executing,9.0,1.0,2.0,43.0,0.0,1.0,11.0,47.0
Released,121.0,3.0,60.0,61.0,4.0,2.0,24.0,26.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,18,19,20
0,Completed,11.0,0.0,0.0,0.0,0.0,0.0,6.0,16.0
1,Executing,9.0,1.0,2.0,43.0,0.0,1.0,11.0,47.0
2,Released,121.0,3.0,60.0,61.0,4.0,2.0,24.0,26.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,18,19,20
0,Completed,11.0,0.0,0.0,0.0,0.0,0.0,6.0,16.0
1,Executing,9.0,1.0,2.0,43.0,0.0,1.0,11.0,47.0
2,Released,121.0,3.0,60.0,61.0,4.0,2.0,24.0,26.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()